使用装饰器实现：
1. 统计函数运行时间
2. 统计函数FPS
3. 注册器
4. 日志(sqlite),登陆尝试机制,登陆锁定(通过 限制在一定时间内失败登陆尝试的次数来保护账户安全)
实现方式：
1. 高阶函数
2. 类
注：
装饰器的本质就是让@后面的东西作为一整个函数去调用被装饰的函数

In [53]:
import time
! pip install pysqlite3
import sqlite3
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
try:
    # init
    cursor.execute('create table user(name varchar(20) primary key, password varchar(20))')
    cursor.execute(r"insert into user values ('wzh','6811')")
    conn.commit()
    print(f"init sqlite")
except:
    print(f"sqlite has been inited")


sqlite has been inited


In [54]:
# 使用高阶函数统计函数运行时长
def funcTimeCounter(func):
    def wrapper(*args,**kwargs):
        start_time=time.time()
        print(f'funcTimeCounter Before is {start_time}')
        result=func(*args,**kwargs)
        end_time=time.time()
        print(f'funcTimeCounter After is {end_time}')
        print(f'funcTimeCounter Diff is {end_time-start_time}')
        return result
    return wrapper
@funcTimeCounter
def testFuncTimeCounter(st:float=0.3):
    time.sleep(st)
    return f'sleep time is {st}'
print(testFuncTimeCounter())

funcTimeCounter Before is 1731326062.1380718


funcTimeCounter After is 1731326062.4381485
funcTimeCounter Diff is 0.3000767230987549
sleep time is 0.3


In [55]:
# 使用高阶函数统计FPS(控制打印信息的频率)
count=0
start_time=None
def funcFpsCounter(func):
    def wrapper(*args,**kwargs):
        global count
        global start_time
        if not start_time:
            start_time=time.time()
        result=func(*args,**kwargs)
        end_time=time.time()
        if end_time-start_time>=1:
            print(f"FPS is {count}")
            count=0
            start_time=end_time
        else:
            count+=1
        return result
    return wrapper
@funcFpsCounter
def testFuncFpsCounter(st:float=0.03):
    time.sleep(st)
for _ in range(100):
    testFuncFpsCounter()
        

FPS is 33
FPS is 33


In [56]:
# 使用高阶函数实现注册器
register=dict()
def funcRegister(key=None):
    def wrapper(func):
        global register
        nonlocal key
        if func:
            if key is None:
                key=func.__name__ 
            register[key]=func
            print(f"{key} registe successfully")
            return func
        else:
            print(f"function is None, can not register")
            return None
    return wrapper

def getFunc(func_name):
    if func_name in register.keys():
        print(f"find {func_name} in register")
        return register[func_name]
    else:
        print(f"can not find {func_name} in register")
        return None

@funcRegister()
def testFuncRegister():
    print('test func register')

print(getFunc('testFuncRegister'))
getFunc('testFuncRegister')()


testFuncRegister registe successfully
find testFuncRegister in register
<function testFuncRegister at 0x709c1c5763e0>
find testFuncRegister in register
test func register


In [57]:
# 使用高阶函数实现
# 1. 日志(sqlite)
# 2. 登陆锁定机制
Fail=dict()

def funcLoginAttemptLimiter(limit=5):
    def decorator(func):
        def wrapper(*args,**kwargs):
            global Fail
            nonlocal limit 
            nonlocal func
            Fail.setdefault(kwargs['name'],0)
            if Fail[kwargs['name']]<limit:
                if func(*args,**kwargs):
                    Fail[kwargs['name']]=0
                    print(f"登陆成功")
                    return True
                else:
                    Fail[kwargs['name']]+=1
                    print(f"登陆失败,密码错误")
                    return False
            else:
                print(f"登陆过于频繁,请稍后重试")
                return False
        return wrapper
    return decorator

@funcLoginAttemptLimiter(limit=3)
def funcLogin(name,password):
    global cursor
    result=cursor.execute(f"select password from user where name='{name}'").fetchone()
    result=str(result[0])
    if result==password:
        return True
    else:
        return False

funcLogin(name='wzh',password='68110')
funcLogin(name='wzh',password='68110')
funcLogin(name='wzh',password='6811')
funcLogin(name='wzh',password='68110')
funcLogin(name='wzh',password='68110')
funcLogin(name='wzh',password='68110')
funcLogin(name='wzh',password='6811')

登陆失败,密码错误
登陆失败,密码错误
登陆成功
登陆失败,密码错误
登陆失败,密码错误
登陆失败,密码错误
登陆过于频繁,请稍后重试


False

In [58]:
# 使用类装饰器
# 统计函数运行时间

class ClsTimeCounter(object):
    def __init__(self,func) -> None:
        self.func=func
    def __call__(self,*args,**kwargs):
        start_time=time.time()
        result=self.func(*args,**kwargs)
        end_time=time.time()
        print(f"time used is {end_time-start_time}")
        return result

@ClsTimeCounter
def testClsTimeCounter(st:float=0.3):
    time.sleep(st)

testClsTimeCounter()

time used is 0.30008864402770996


In [59]:
# 使用类装饰器统计函数FPS
class ClsFpsCounter(object):
    def __init__(self,func):
        self.func=func
        self.start_time=None
        self.end_time=None
        self.count=0
    def __call__(self,*args,**kwargs):
        if self.start_time is None:
            self.start_time=time.time()
        result=self.func(*args,**kwargs)
        self.end_time=time.time()
        if self.end_time-self.start_time>=1:
            print(f"FPS is {self.count}")
            self.start_time=self.end_time
            self.count=0
        else:
            self.count+=1
        return result

@ClsFpsCounter
def testClsFpsCounter(st:float=0.3):
    time.sleep(st)

for _ in range(10):
    testClsFpsCounter()

FPS is 3
FPS is 3


In [60]:
# 使用类装饰器实现注册器
class ClsRegister(object):
    register=dict()
    def __init__(self):
        pass
    @classmethod
    def registe(cls,func_name=None):
        def decorator(func):
            nonlocal func_name
            if func_name is None:
                func_name=func.__name__
            cls.register[func_name]=func
        return decorator
    @classmethod
    def getFunc(cls,func_name):
        return cls.register[func_name]

@ClsRegister.registe()
def testClsRegister():
    print(f'test class register')
    return  True

print(ClsRegister.getFunc('testClsRegister')())

test class register
True


In [61]:
# 使用类装饰器实现
# 日志(sqlite) 尝试登陆机制

class ClsLoginAttemptLimiter(object):
    Fail=dict()
    def __init__(self,limit=5):
        self.limit=limit
    def __call__(self,func):
        def wrapper(*args,**kwargs):
            self.Fail.setdefault(kwargs['name'],0)
            if self.Fail[kwargs['name']]>self.limit:
                print(f"登陆过于频繁,请稍后重试")
                return False
            else:
                if func(*args,**kwargs):
                    self.Fail[kwargs['name']]=0
                    print(f"登陆成功")
                else:
                    self.Fail[kwargs['name']]+=1
                    print(f"登陆失败,密码错误")
        return wrapper

@ClsLoginAttemptLimiter(limit=3)
def clsLogin(name,password):
    global cursor
    result=cursor.execute(f"select password from user where name='{name}'").fetchone()
    result=str(result[0])
    if result==password:
        return True
    else:
        return False

clsLogin(name='wzh',password='68110')
clsLogin(name='wzh',password='68110')
clsLogin(name='wzh',password='6811')
clsLogin(name='wzh',password='68110')
clsLogin(name='wzh',password='68110')
clsLogin(name='wzh',password='68110')
clsLogin(name='wzh',password='6811')

登陆失败,密码错误
登陆失败,密码错误
登陆成功
登陆失败,密码错误
登陆失败,密码错误
登陆失败,密码错误
登陆成功
